In [2]:
import cv2
import json
from datetime import timedelta
import os

In [5]:
def convert_seconds_to_timecode(seconds):
    # Função auxiliar para converter segundos para o formato HH:MM:SS
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [27]:
def detect_logo_in_video(video_path, logo_path, threshold=0.5):
    # Carregar vídeo e logo
    print(video_path)
    print(threshold)
    cap = cv2.VideoCapture(video_path)
    logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)

    # Verificar se o vídeo foi carregado corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo")
        return

    # Obter FPS e total de frames para calcular o timestamp
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Variáveis de controle
    start_time = None
    end_time = None
    logo_detected = False
    detection_results = []

    # Processar o vídeo frame a frame
    for frame_idx in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Converter frame para grayscale (escala de cinza)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Aplicar template matching
        result = cv2.matchTemplate(gray_frame, logo, cv2.TM_SQDIFF_NORMED)
        min_val, _, min_loc, _ = cv2.minMaxLoc(result)

        # Verificar se a logo foi detectada
        if min_val < threshold:
            if not logo_detected:
                # Primeira detecção
                logo_detected = True
                start_time = frame_idx / fps
        else:
            if logo_detected:
                # Detecção de desaparecimento
                logo_detected = False
                end_time = frame_idx / fps

                # Salvar o intervalo de tempo da detecção
                detection_results.append({
                    "start": convert_seconds_to_timecode(start_time),
                    "end": convert_seconds_to_timecode(end_time)
                })

                # Resetar as variáveis
                start_time = None
                end_time = None

    # Caso a logo tenha sido detectada até o final do vídeo
    if logo_detected:
        end_time = frame_count / fps
        detection_results.append({
            "start": convert_seconds_to_timecode(start_time),
            "end": convert_seconds_to_timecode(end_time)
        })

    # Salvar os resultados em um arquivo JSON
    #with open(output_json_path, 'w') as json_file:
        #json.dump(detection_results, json_file, indent=4)

    cap.release()
    print(detection_results)

In [6]:
def detect_logo_in_video(video_path, logo_path, output_folder="tm_sqdiff_frames", threshold=0.2):
    # Carregar vídeo e logo
    cap = cv2.VideoCapture(video_path)
    logo = cv2.imread(logo_path, cv2.IMREAD_GRAYSCALE)

    # Verificar se o vídeo foi carregado corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Obter FPS e total de frames para calcular o timestamp
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Dimensões da logo
    logo_w, logo_h = logo.shape[::-1]

    # Criar a pasta de saída se não existir
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Variáveis de controle
    start_time = None
    end_time = None
    logo_detected = False
    detection_results = []

    # Processar o vídeo frame a frame
    for frame_idx in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Converter frame para grayscale (escala de cinza)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Aplicar template matching
        result = cv2.matchTemplate(gray_frame, logo, cv2.TM_SQDIFF_NORMED)
        min_val, _, min_loc, _ = cv2.minMaxLoc(result)

        # Verificar se a logo foi detectada
        if min_val < threshold:
            if not logo_detected:
                # Primeira detecção
                logo_detected = True
                start_time = frame_idx / fps

            # Desenhar o retângulo ao redor da logo detectada
            top_left = min_loc
            bottom_right = (top_left[0] + logo_w, top_left[1] + logo_h)
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)

        else:
            if logo_detected:
                # Detecção de desaparecimento
                logo_detected = False
                end_time = frame_idx / fps

                # Salvar o intervalo de tempo da detecção
                detection_results.append({
                    "start": convert_seconds_to_timecode(start_time),
                    "end": convert_seconds_to_timecode(end_time)
                })

                # Resetar as variáveis
                start_time = None
                end_time = None

        # Salvar o frame como imagem
        frame_filename = os.path.join(output_folder, f"frame_{frame_idx:05d}_direito.jpg")
        cv2.imwrite(frame_filename, frame)

    # Caso a logo tenha sido detectada até o final do vídeo
    if logo_detected:
        end_time = frame_count / fps
        detection_results.append({
            "start": convert_seconds_to_timecode(start_time),
            "end": convert_seconds_to_timecode(end_time)
        })

    cap.release()

    # Retornar os resultados
    return detection_results


In [7]:
video_path = "../videos/15s-canto-direito.mp4"
logo_path = "../target/logo_target.png"
detect_logo_in_video(video_path, logo_path)

[]